## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-08-08-56-16 +0000,bbc,Netanyahu divides Israelis and allies with pla...,https://www.bbc.com/news/articles/cj4w2q9k4pjo...
1,2025-08-08-08-56-01 +0000,nypost,Nancy Pelosi says sex changes for trans kids i...,https://nypost.com/2025/08/08/us-news/nancy-pe...
2,2025-08-08-08-49-31 +0000,nypost,Israel approves Netanyahu’s plans to take cont...,https://nypost.com/2025/08/08/world-news/israe...
3,2025-08-08-08-43-50 +0000,nyt,Israeli Security Cabinet Approves Military Tak...,https://www.nytimes.com/2025/08/07/world/middl...
4,2025-08-08-08-40-37 +0000,bbc,US shrug at Israel's Gaza escalation at odds w...,https://www.bbc.com/news/articles/cn92je014dyo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2600/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
109,trump,74
8,gaza,18
5,new,16
946,tariffs,16
622,will,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
121,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...,125
188,2025-08-07-19-02-30 +0000,wapo,Here’s where the money from Trump’s tariffs wi...,https://www.washingtonpost.com/business/2025/0...,120
259,2025-08-07-14-30-28 +0000,wapo,Trump’s new tariffs are hitting these major tr...,https://www.washingtonpost.com/business/2025/0...,119
171,2025-08-07-20-10-22 +0000,wapo,Trump celebrates new tariffs as negotiations c...,https://www.washingtonpost.com/business/2025/0...,119
292,2025-08-07-11-54-12 +0000,bbc,Trump's sweeping new tariffs take effect again...,https://www.bbc.com/news/articles/cx23jmvn5yzo...,118


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
121,125,2025-08-07-21-50-00 +0000,wsj,President Trump said he would nominate Stephen...,https://www.wsj.com/politics/policy/stephen-mi...
45,67,2025-08-08-03-37-58 +0000,wapo,Israel’s security cabinet approves plan for mi...,https://www.washingtonpost.com/world/2025/08/0...
198,60,2025-08-07-18-30-00 +0000,wsj,Sen. John Cornyn said the FBI will help track ...,https://www.wsj.com/politics/policy/fbi-texas-...
214,50,2025-08-07-17-44-51 +0000,nypost,"Skydance CEO David Ellison feels the heat, mel...",https://nypost.com/2025/08/07/media/skydance-c...
79,41,2025-08-08-00-24-53 +0000,bbc,"Dozens of migrants detained under 'one in, one...",https://www.bbc.com/news/articles/ce35v0zyzvlo...
101,39,2025-08-07-22-40-47 +0000,nypost,Disney settles Gina Carano’s lawsuit after fir...,https://nypost.com/2025/08/07/media/disney-set...
213,37,2025-08-07-17-51-00 +0000,wsj,President Trump’s tariff rollout has taken num...,https://www.wsj.com/economy/trade/trump-tariff...
169,34,2025-08-07-20-17-49 +0000,latimes,California Supreme Court sides with environmen...,https://www.latimes.com/environment/story/2025...
158,33,2025-08-07-20-52-50 +0000,nyt,Staggering U.S. Tariffs Begin as Trump Widens ...,https://www.nytimes.com/2025/08/07/business/ec...
109,32,2025-08-07-22-20-42 +0000,nypost,Trusted Biden aide testifies ex-prez skipped S...,https://nypost.com/2025/08/07/us-news/trusted-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
